[View in Colaboratory](https://colab.research.google.com/github/sl2820/Deep_Learning_Study/blob/master/RNN_Practice.ipynb)

# Sequence Classification


In [1]:
import tensorflow as tf
import tensorflow.examples.tutorials.mnist.input_data as input_data
import tensorflow.contrib.rnn as rnn
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
print ("Packages Loaded")

Packages Loaded


### Get MNIST


In [2]:
mnist = input_data.read_data_sets("data/", one_hot = True)
trainimgs, trainlabels, testimgs, testlabels = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels
ntrain, ntest, dim, nclasses \
= trainimgs.shape[0], testimgs.shape[0], trainimgs.shape[1], trainlabels.shape[1]
print("MNIST LOADED")
print ("TF Version is: %s" % (tf.__version__))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
MN

### Define Model
LSTM 자체는 텐서플로우 것 사용, Input 과 Output에 대한 것만 Define 하면 됌.

In [0]:
diminput = 28
dimhidden = 128
dimoutput = nclasses
nsteps = 28
weights = {
    'hidden': tf.Variable(tf.random_normal([diminput, dimhidden])), #28 --> 128
    'out': tf.Variable(tf.random_normal([dimhidden, dimoutput]))  # 128 --> 10
}

biases = {
    'hidden': tf.Variable(tf.random_normal([dimhidden])),
    'out': tf.Variable(tf.random_normal([dimoutput]))
}

### Define Function

In [9]:
def _RNN(_X, _istate,_W,_b,_nsteps,_name):
  # 1. Permute input from [batchsize, nsteps, diminput] --> [nsteps, batchsize, diminput]
  # Current _X = 784 dim
  _X = tf.transpose(_X, [1,0,2])   
  
  # 2. Reshape input to [nsteps*batchsize, diminput]
  _X = tf.reshape(_X, [-1, diminput])
  
  # 3. Input to hidden layer
  _H = tf.matmul(_X, _W['hidden']) + _b['hidden']
  
  # 4. Split data to 'nsteps' chunks --> LIST
  _Hsplit = tf.split(_H, _nsteps, axis=0)
  # 5. Get LSTM's final output (_LSTM_O) and State (_LSTM_S)
  with tf.variable_scope(_name):
    # RNN <-- tf.contrib.rnn
    lstm_cell = rnn.BasicLSTMCell(dimhidden, forget_bias=1.0, state_is_tuple=False, reuse=tf.AUTO_REUSE)
    _LSTM_O, _LSTM_S = rnn.static_rnn(lstm_cell, _Hsplit, initial_state=_istate)
    
  _O = tf.matmul(_LSTM_O[-1], _W['out']) + _b['out']  #_LSTM_O[-1] --> 맨 마지막 아웃풋
  return{
      'X': _X, 'H': _H, 'Hsplit': _Hsplit, 'LSTM_O': _LSTM_O, 'LSTM_S': _LSTM_S, 'O': _O
  }

print("Function Ready")

Function Ready


### Define Graph

In [10]:
learning_rate = 0.001
x = tf.placeholder("float", [None, nsteps, diminput])
istate = tf.placeholder("float", [None, 2*dimhidden])
y = tf.placeholder("float", [None, dimoutput])
myrnn = _RNN(x, istate, weights, biases, nsteps, 'basic')
pred = myrnn['O']
celoss = tf.nn.softmax_cross_entropy_with_logits
cost = tf.reduce_mean(celoss(logits=pred,labels = y))
optm = tf.train.AdamOptimizer(learning_rate).minimize(cost)
accr = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(pred,1), tf.argmax(y,1)), tf.float32))
init = tf.global_variables_initializer()
print ("Network Ready")

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Network Ready


### Run

In [0]:
t_epochs = 5
batch_size = 128
display_step = 1
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
sess.run(init)

### Optimization

In [17]:
#Optimization Starts
for epoch in range(t_epochs):
  avg_cost = 0.
  total_batch = int(mnist.train.num_examples/batch_size)
  for i in range(total_batch):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    batch_xs = batch_xs.reshape((batch_size,nsteps,diminput))
    feeds = {x: batch_xs, y: batch_ys, istate: np.zeros((batch_size, 2*dimhidden))}
    sess.run(optm, feed_dict=feeds)
    avg_cost += sess.run(cost, feed_dict= feeds)/total_batch
    
  if epoch % display_step ==0:
    print ("Epoch: %03d/%03d Cost: %.9f" % (epoch+1, t_epochs, avg_cost))
    feeds = {x: batch_xs, y: batch_ys, istate: np.zeros((batch_size, 2*dimhidden))}
    train_acc = sess.run(accr, feed_dict=feeds)
    print("Train Accuracy: %.3f" %(train_acc))
    testimgs = testimgs.reshape((ntest,nsteps,diminput))
    feeds = {x: testimgs, y: testlabels, istate: np.zeros((ntest, 2*dimhidden))}
    test_acc = sess.run(accr,feed_dict = feeds)
    print("Test Accuracy: %.3f" % (test_acc))

print("Optimization Done")

Epoch: 001/005 Cost: 0.021602890
Train Accuracy: 0.992
Test Accuracy: 0.980
Epoch: 002/005 Cost: 0.016210595
Train Accuracy: 0.992
Test Accuracy: 0.981
Epoch: 003/005 Cost: 0.016350848
Train Accuracy: 1.000
Test Accuracy: 0.982
Epoch: 004/005 Cost: 0.015117944
Train Accuracy: 1.000
Test Accuracy: 0.983
Epoch: 005/005 Cost: 0.013567779
Train Accuracy: 1.000
Test Accuracy: 0.982
Optimization Done


 *여기까지가 RNN 8:27까지 한것  (10/9)* 

### What if we use smaller number of Sequences?

In [19]:
#( Providing only the early [24,25,26,27]개 images로만) 학습할 경우
for _nsteps in [24,25,26,27,28]:
  # Test with truncated Sequences
  testimgs = testimgs.reshape((ntest,nsteps,diminput))
  testimgs_truncated = np.zeros(testimgs.shape)
  testimgs_truncated[:,28-_nsteps:] = testimgs[:, :_nsteps,:] 
  feeds = {x: testimgs_truncated, y: testlabels, istate: np.zeros((ntest, 2*dimhidden))}
  test_acc = sess.run(accr, feed_dict=feeds)
  print ("With [%d] Sequences, Test Accuracy is [%.3f]" % (_nsteps, test_acc)) 

With [24] Sequences, Test Accuracy is [0.758]
With [25] Sequences, Test Accuracy is [0.876]
With [26] Sequences, Test Accuracy is [0.957]
With [27] Sequences, Test Accuracy is [0.979]
With [28] Sequences, Test Accuracy is [0.982]


# What is Happening Inside the RNN
Step by step demonstration


### 1. Inputes to the RNN

In [21]:
# Size of Initial Inputs (기본 MNIST)
batch_size = 5
xtest, _ = mnist.test.next_batch(batch_size)
print ("Shape of 'xtest' is %s" %(xtest.shape,))

Shape of 'xtest' is (5, 784)


### 2. Reshaped Inputs

In [22]:
# We create into 28*28 with 5 channels
xtest_reshape = xtest.reshape((batch_size, nsteps, diminput))
print ("Batch size is: [%d]" % (batch_size))
print ("Shape of 'xtest_reshape' is %s" % (xtest_reshape.shape,))

Batch size is: [5]
Shape of 'xtest_reshape' is (5, 28, 28)


### 3. Feeds

In [0]:
feeds = {x: xtest_reshape, istate: np.zeros((batch_size, 2*dimhidden))}

### 4. Individual Input to the LSTM

In [25]:
# Size that going into LSTM [5*28, 28] --> 5개의 Data가 있고, 각각 5개의 데이터가 28개의 Sequence가 있다
# 그리고 각 1개의 데이터 Dimension이 28
rnnoutput_X = sess.run(myrnn['X'], feed_dict=feeds)
print ("Shape of 'rnnoutput_X' is %s" % (rnnoutput_X.shape,))

Shape of 'rnnoutput_X' is (140, 28)


### 5. Intermediate State

In [26]:
# Size at the Hidden Layer 각 데이터 Dimension을 28 --> 128 확장
rnnoutput_H = sess.run(myrnn['H'], feed_dict=feeds)
print ("Shape of 'rnnoutput_H' is %s" % (rnnoutput_H.shape,))

Shape of 'rnnoutput_H' is (140, 128)


### 6. Actual Input to LSTM (LIST)

In [28]:
# Hsplit을 이용해서 140을 28로 만들어 리스트가 됨. 리스트안에 각각의 아이템은 5*128
rnnoutput_Hsplit = sess.run(myrnn['Hsplit'], feed_dict=feeds)
print ("Type of 'rnnoutput_Hsplit' is %s" % (type(rnnoutput_Hsplit)))
print ("Length of 'rnnoutput_Hsplit' is %s and the shape of each item is %s"
      % (len(rnnoutput_Hsplit), rnnoutput_Hsplit[0].shape))

Type of 'rnnoutput_Hsplit' is <class 'list'>
Length of 'rnnoutput_Hsplit' is 28 and the shape of each item is (5, 128)


### Output from the LSTM (LIST)

In [29]:
# 들어간 사이즈와 동일하게 Out / 28개의 Output으로, 각각 5*128
rnnoutput_LSTM_O = sess.run(myrnn['LSTM_O'], feed_dict=feeds)
print ("Type of 'rnnoutput_LSTM_O' is %s" % (type(rnnoutput_LSTM_O)))
print ("Length of 'rnnoutput_LSTM_O' is %s and the shape of each item is %s"
      % (len(rnnoutput_LSTM_O), rnnoutput_LSTM_O[0].shape))

Type of 'rnnoutput_LSTM_O' is <class 'list'>
Length of 'rnnoutput_LSTM_O' is 28 and the shape of each item is (5, 128)


### Final Prediction

In [30]:
# 5개의 아이템에 대해서 10개의 클래스로
rnnoutput_O = sess.run(myrnn['O'], feed_dict=feeds)
print ("Shapeof rnnoutput_O is %s" % (rnnoutput_O.shape,))

Shapeof rnnoutput_O is (5, 10)
